<a href="https://colab.research.google.com/github/ErickMM98/NetworkAnalysis_TwitterTrends/blob/main/colab_burstkit_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download all datasets and codes

In [1]:
#@title Dataset

!wget https://zenodo.org/record/4686013/files/virality2013.tar.gz
!tar -xf virality2013.tar.gz

--2022-04-25 00:15:28--  https://zenodo.org/record/4686013/files/virality2013.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 526346240 (502M) [application/octet-stream]
Saving to: ‘virality2013.tar.gz’

virality2013.tar.gz 100%[===================>] 501.96M  4.88MB/s    in 1m 48s  

2022-04-25 00:17:17 (4.67 MB/s) - ‘virality2013.tar.gz’ saved [526346240/526346240]



In [19]:
#@title clone repository
!git clone https://github.com/ErickMM98/NetworkAnalysis_TwitterTrends.git
!mv /content/NetworkAnalysis_TwitterTrends/burstkit /content/

fatal: destination path 'NetworkAnalysis_TwitterTrends' already exists and is not an empty directory.
mv: cannot stat '/content/NetworkAnalysis_TwitterTrends/burstkit': No such file or directory


# Project


In [28]:
import burstkit
import burstkit.util as util
from burstkit.util import read_files

read_files.is_path_file_in_data_path("/content/follower_gcc.anony.dat")

False